## 임의의 조건 탐색 및 데이터 수정

`DataFrame[조건식]`

In [1]:
import pandas as pd

In [2]:
students = pd.DataFrame({
    '이름': ['장화', '홍련', '콩쥐', '팥쥐', '해님', '달님'],
    '국어': [70, 85, None, 100, None, 85],
    '수학': [65, 100, 80, 95, None, 70]
})
students[students['이름'] == '장화']

,이름,국어,수학
0,장화,70.0,65.0


<hr>

`DataFrame[(조건1) & (조건2) & ((조건3) | (조건4))]`

In [3]:
students[(students['국어'] >= 80) & (students['수학'] >= 80)]

,이름,국어,수학
1,홍련,85.0,100.0
3,팥쥐,100.0,95.0


<hr>

`df.loc[조건문, '추가/변경할 컬럼명']`
- '추가/변경할 컬럼명'이 존재하는 컬럼인 경우 조건문에 부합하는 행의 '추가/변경할 컬럼명'의 값을 변경
- '추가/변경할 컬럼명'이 존재하지 않는 컬럼인경우 새로운 컬럼을 만들고 조건문에 부합하는 행에는 지정된 값을, 그 외의 행에는 NaN을 삽입

In [4]:
students.loc[6, '이름':'수학'] = ['별님', 50, 60]
students.loc[(students['국어'] >= 80)&(students['수학'] >= 70), '합격'] = 'Pass'
students.loc[students['합격'] != 'Pass', '합격'] = 'Fail'
students

,이름,국어,수학,합격
0,장화,70.0,65.0,Fail
1,홍련,85.0,100.0,Pass
2,콩쥐,NaN,80.0,Fail
3,팥쥐,100.0,95.0,Pass
4,해님,NaN,NaN,Fail
5,달님,85.0,70.0,Pass
6,별님,50.0,60.0,Fail


<hr>

하나의 컬럼에 여러 개의 조건에 따라 다른 값을 반환해야하는 경우 `np.select(조건목록, 선택목록, default=디폴트값)`

In [5]:
import numpy as np

# 조건목록
condition_list = [
    (students['국어'] >= 90),
    (students['국어'] >= 80) & (students['국어'] < 90), 
    (students['국어'] >=70 & (students['국어'] < 80))
]

# 선택목록
choice_list = ['A', 'B', 'C']

students['점수'] = np.select(condition_list, choice_list, default='F')
students

,이름,국어,수학,합격,점수
0,장화,70.0,65.0,Fail,C
1,홍련,85.0,100.0,Pass,B
2,콩쥐,NaN,80.0,Fail,F
3,팥쥐,100.0,95.0,Pass,A
4,해님,NaN,NaN,Fail,F
5,달님,85.0,70.0,Pass,B
6,별님,50.0,60.0,Fail,C


<hr>

## 결측값 탐색 및 수정

### 결측값

In [6]:
students.isna()

,이름,국어,수학,합격,점수
0,False,False,False,False,False
1,False,False,False,False,False
2,False,True,False,False,False
3,False,False,False,False,False
4,False,True,True,False,False
5,False,False,False,False,False
6,False,False,False,False,False


### 결측이 아닌 값

In [7]:
students.notna()

,이름,국어,수학,합격,점수
0,True,True,True,True,True
1,True,True,True,True,True
2,True,False,True,True,True
3,True,True,True,True,True
4,True,False,False,True,True
5,True,True,True,True,True
6,True,True,True,True,True


In [8]:
students.isna().sum() # True는 1, False는 0과 대치 됨

이름    0
국어    2
수학    1
합격    0
점수    0
dtype: int64

In [9]:
students.isna().sum(1) # 행 합계

0    0
1    0
2    1
3    0
4    2
5    0
6    0
dtype: int64

### 결측값 제거

`DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplase=False)`
- axis: 0 또는 'index'이면 결측값이 포함된 행을 삭제하고, 1 또는 'columns'이면 결측값이 포함된 열을 삭제
- how: 'any'이면 결츨값이 존재하는 모든 행/열 삭제, 'all'이면 모든 값이 결측값일 때 삭제
- thresh: 정숫값을 지정하면 결측값이 아닌 값이 그보다 많을 때 행 또는 열을 유지
- subset: 어떤 레이블에 결측값이 존재하면 삭제할지 정의
- inplace: True이면 제자리에서 작없을 수행하고 None을 반환

In [10]:
students.dropna() # 결측값이 포함된 모든 행을 삭제

,이름,국어,수학,합격,점수
0,장화,70.0,65.0,Fail,C
1,홍련,85.0,100.0,Pass,B
3,팥쥐,100.0,95.0,Pass,A
5,달님,85.0,70.0,Pass,B
6,별님,50.0,60.0,Fail,C


In [11]:
students.dropna(thresh=4) # 결측값이 아닌 값이 4개보다 많은 행만 남기기

,이름,국어,수학,합격,점수
0,장화,70.0,65.0,Fail,C
1,홍련,85.0,100.0,Pass,B
2,콩쥐,NaN,80.0,Fail,F
3,팥쥐,100.0,95.0,Pass,A
5,달님,85.0,70.0,Pass,B
6,별님,50.0,60.0,Fail,C


### 결측값 대체

`DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None)`
- value: 단일 값 혹은 dict/Series/DataFrame 형식으로 대체할 값을 직접 입력
- method:'pad', 'ffill' 이전 값으로 채우고, 'backfil'/'bfill'은 다음에 오는 값으로 채움
- axis: 0 또는 'index'이면 행 방향으로 채우고, 1 또는 'columns'이면 열 방향으로 채움
- limit: method 인자를 지정한 경우 limit으로 지정한 개수만큼만 대체할 수 있음

In [12]:
health = pd.DataFrame({
    '연도': [2017, 2018, 2019, 2020, 2021, 2022],
    '키': [160, 162, 165, None, None, 166],
    '몸무게': [53, 52, None, 50, 51, 54],
    '시력': [1.2, None, 1.2, 1.2, 1.1, 0.8],
    '병결': [None, None, None, 2, None, 1]
})
health.fillna(0) # 결측값을 0으로 대체

,연도,키,몸무게,시력,병결
0,2017,160.0,53.0,1.2,0.0
1,2018,162.0,52.0,0.0,0.0
2,2019,165.0,0.0,1.2,0.0
3,2020,0.0,50.0,1.2,2.0
4,2021,0.0,51.0,1.1,0.0
5,2022,166.0,54.0,0.8,1.0


In [13]:
health.fillna(health.mean()) # 결측값을 평균으로 대체

,연도,키,몸무게,시력,병결
0,2017,160.00,53.0,1.2,1.5
1,2018,162.00,52.0,1.1,1.5
2,2019,165.00,52.0,1.2,1.5
3,2020,163.25,50.0,1.2,2.0
4,2021,163.25,51.0,1.1,1.5
5,2022,166.00,54.0,0.8,1.0


In [15]:
health['병결'] = health['병결'].fillna(0)
health['몸무게'] = health['몸무게'].fillna(health['몸무게'].mean())
health

,연도,키,몸무게,시력,병결
0,2017,160.0,53.0,1.2,0.0
1,2018,162.0,52.0,NaN,0.0
2,2019,165.0,52.0,1.2,0.0
3,2020,NaN,50.0,1.2,2.0
4,2021,NaN,51.0,1.1,0.0
5,2022,166.0,54.0,0.8,1.0
